In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications.resnet50 import ResNet50

from keras.layers.pooling import MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D
from keras.models import Model

In [2]:
BATCH_SIZE = 64
img_height = 224
img_width = 224

In [3]:
train_data_dir = '/kaggle/input/car-train/train'
test_data_dir = '/kaggle/input/car-valid/valid'

# InceptionV3

In [8]:
from keras.applications.inception_v3 import preprocess_input
train_generator = ImageDataGenerator(rescale=1.0/255,
                                     rotation_range=20, 
                                     brightness_range=[0.1, 1.0],
                                     width_shift_range=0.1, 
                                     height_shift_range=0.1,
                                     horizontal_flip=True, 
                                    #  shear_range = 0.2,
                                     zoom_range = [0.2,1],
                                     preprocessing_function=preprocess_input
                                     ) 

test_generator = ImageDataGenerator(rescale=1.0/255,
                                    preprocessing_function=preprocess_input
                                    )

In [9]:
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(299, 299),
                                               class_mode='categorical',
                                               batch_size=BATCH_SIZE, )

validgen = test_generator.flow_from_directory(test_data_dir,
                                               target_size=(299, 299),
                                               class_mode='categorical',
                                               batch_size=BATCH_SIZE,)

Found 6515 images belonging to 196 classes.
Found 1629 images belonging to 196 classes.


In [4]:
def fine_tune(base_model,num_classes,dropout):
    x=base_model.output
    x = AveragePooling2D((5, 5), name='avg_pool')(x)
    x = Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = Dropout(dropout)(x)

    predictions = layers.Dense(num_classes, activation='softmax', name='finalfc')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
#     model.summary()
    return model

In [11]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
HEIGHT = 299
WIDTH = 299

input_shape=(HEIGHT, WIDTH, 3)

lr = 0.0001
dropout = 0.7
epochs = 100

base_model = InceptionV3(weights='imagenet',
                            include_top=False,
                            input_shape=input_shape)

model = fine_tune(base_model,num_classes,dropout=dropout)
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=lr), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [12]:
train_steps = len(traingen.filenames) // BATCH_SIZE
validation_steps = len(validgen.filenames) // BATCH_SIZE

In [13]:
print(train_steps)
print(validation_steps)

101
25


In [15]:
inc_history = model.fit_generator(generator=traingen,
                                        validation_data=validgen,
                                        steps_per_epoch=train_steps,
                                        epochs=100,verbose=2,validation_steps=validation_steps)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2023-02-10 09:59:07.679756: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2023-02-10 09:59:20.740273: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


101/101 - 228s - loss: 5.3008 - accuracy: 0.0060 - val_loss: 5.2806 - val_accuracy: 0.0056
Epoch 2/100
101/101 - 181s - loss: 5.2191 - accuracy: 0.0116 - val_loss: 5.2674 - val_accuracy: 0.0075
Epoch 3/100
101/101 - 176s - loss: 5.0271 - accuracy: 0.0226 - val_loss: 4.8773 - val_accuracy: 0.0487
Epoch 4/100
101/101 - 179s - loss: 4.7993 - accuracy: 0.0426 - val_loss: 4.4204 - val_accuracy: 0.0862
Epoch 5/100
101/101 - 185s - loss: 4.4562 - accuracy: 0.0800 - val_loss: 3.9301 - val_accuracy: 0.1281
Epoch 6/100
101/101 - 175s - loss: 4.0906 - accuracy: 0.1234 - val_loss: 3.3572 - val_accuracy: 0.2587
Epoch 7/100
101/101 - 203s - loss: 3.7141 - accuracy: 0.1685 - val_loss: 3.1297 - val_accuracy: 0.2575
Epoch 8/100
101/101 - 172s - loss: 3.3792 - accuracy: 0.2257 - val_loss: 2.7384 - val_accuracy: 0.3225
Epoch 9/100
101/101 - 184s - loss: 3.0484 - accuracy: 0.2775 - val_loss: 2.3780 - val_accuracy: 0.3994
Epoch 10/100
101/101 - 170s - loss: 2.8012 - accuracy: 0.3254 - val_loss: 2.0110 - va

In [16]:
model.save('inception_model.h5')

In [17]:
validation_score = model.evaluate_generator(validgen, steps=validation_steps)
print('Validation loss: ', validation_score[0])
print('Validation acc:  ', validation_score[1])

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Validation loss:  1.9570105075836182
Validation acc:   0.7131249904632568


In [1]:
def plot_loss_acc(history):
    plt.figure(figsize=(20,7))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'][1:])    
    plt.plot(history.history['val_loss'][1:])    
    plt.title('model loss')    
    plt.ylabel('val_loss')    
    plt.xlabel('epoch')    
    plt.legend(['Train','Validation'], loc='upper left')
    
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'][1:])
    plt.plot(history.history['val_accuracy'][1:])
    plt.title('Model Accuracy')
    plt.ylabel('val_acc')
    plt.xlabel('epoch')
    plt.legend(['Train','Validation'], loc='upper left')
    plt.show()